In [ ]:
%matplotlib inline
import pysal as ps
import numpy as np
import pandas as pd
import descartes
import geopandas as gp
import shapely
import copy
import random
import os
import sys
import acs_regionalization as ACS
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 30,30
rcParams['patch.linewidth'] = 0.5  #set default polygon line width

In [ ]:
shp_map = gp.read_file('LA_County.shp')
shp_map['constant'] = 1
p = shp_map.plot('constant', alpha=0.5, cmap='Reds', edgecolor="b")
p.axis('off')
p.plot()

In [ ]:
w = ps.rook_from_shapefile('LA_County.shp', idVariable='GEOID')
print(w.n)

In [ ]:
# summary of the connectivity structure of the Los Angeles County census tracts
w_hist_data = np.array(w.histogram)
fig, ax = plt.subplots(figsize=(10,10))
ax.bar(w_hist_data[:,0], w_hist_data[:,1], align='center', width=1)
ax.set_xlabel('number of neighbors')
ax.set_ylabel('number of tracts')

In [ ]:
# read in list of census tracts to exclude
# using arcmap to view zoning data overlaid on census tracts symbolized by total population estimate per tract
# exclude = open('excluded_lacounty.csv')
# exclude = exclude.readline()
# exclude = exclude.replace('g','')
# exclude = exclude.split(',')
# print exclude

In [ ]:
# read in the ACS estimates and standard errors
population = pd.read_csv('ACS_Full_Table.csv', index_col=0)
# csv file of estimated total veteran population by census tract.
est_vet_pop = pd.read_csv('vet_est_lac2.csv', index_col=0)
# standard error csv by census tract.
# stderr_pop = pd.read_csv('stdErr_lactracts.csv', index_col=0)
# margin of error calculation 
# moe_pop = stderr_pop * 1.65  # changed from 1.645 to 1.65 to reduce decimal places in analysis
moe_pop = pd.read_csv('moe_pop.csv', index_col=0)
# read the census tract ids into a python object 
pop_tract_ids = open('tract_ids.csv')
# split the csv into a python list object
pop_tract_ids = pop_tract_ids.readline().split(',')
# calculate the estimate ratio, standard error ratio, and margin of error for each census tract as a python object
est_ratio = pd.read_csv('est_ratio2.csv')
# std_ratio = (moe_pop/1.65)
# print (est_ratio.dtypes)
moe_ratio = moe_pop

# population_ratio.csv is a CSV file with the census tract id and the above ratio names as columns.
ratio_names = open('population_ratio.csv')
ratio_names = ratio_names.readline().split(',')

In [ ]:
# depending on what the machine size of the Virtual Machine, the algorthim can take several minutes to complete.
# i am currently using a first generation Google Cloud Platform Compute Engine Virtual Machine size n1-standard-4.
# this machine has 4 virtual CPU cores and 8gb of RAM.

# open the shapefile of the census tracts.
shp = ps.open('LA_County.shp')
# using a random seed in python generates a random number and using the same seed value ensures that the same results are generated.
# if a random seed is not used, I cant reliably check that the same result comes from each execution of the code, 
# since the code uses pseudorandom numbers in the ACS Regionalization algorythm.
              
random.seed(745)     # to ensure we get the same solution each time
np.random.seed(745)  # to ensure we get the same solution each time
results = ACS.ACS_Regions(w=w,\
                          target_est_prop=est_vet_pop.values,\
                          target_moe_prop=moe_pop.values,\
                          target_est_ratio=est_ratio.values,\
                          target_moe_ratio=moe_ratio.values,\
                          count_est=population.values,\
                          target_th_all=0.05,\
                          compactness=shp)
print 'regionalization finished'

In [ ]:
# ratio and population data is organized as numerator-denominator pairs
print ratio_names
print est_ratio.columns  # names of the numerators and denominators
print '\nthe numerator of %s is %s, and the denominator is %s'%(ratio_names[0], population.columns[4], population.columns[7])
print 'the numerator of %s is %s, and the denominator is %s'%(ratio_names[1], population.columns[4], population.columns[2])

print (est_ratio.values)
print (est_vet_pop.values)

In [ ]:
# some basic diagnostics on the solution
print 'number of regions:', len(results.regions)
print 'total time (seconds):', results.time['total']

In [ ]:
# This is a python function to create the side by side plots of LA County

# We'll make a few side-by-side plots. This code wraps the setup into a function. This will plot out the two maps
# of LA County before and after regionalization.
def two_plot_setup():
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(40,40))
    axes[0].set_aspect('equal')
    axes[1].set_aspect('equal')
    axes[0].axis('off')
    axes[1].axis('off')
    return fig, axes

In [ ]:
#This cell calls the above function and actually creates the two plots.

fig, axes = two_plot_setup()

# color tracts based on region ID  #This shows all the 
rids = copy.copy(results.region_ids)
shp_map['rids'] = rids
shp_map.plot('rids', categorical=True, alpha=1, colormap='Dark2', axes=axes[0])

# disolve the tracts into regions
region_groups = shp_map.groupby('rids')
region_map = gp.GeoDataFrame(index=region_groups.indices.keys())
region_map['rids'] = region_groups.indices.keys()
region_map['geometry'] = region_groups.geometry.apply(shapely.ops.unary_union)
region_map.plot('rids', categorical=True, alpha=1, colormap='Dark2', axes=axes[1])